In [1]:
from sys import path
from os import listdir, path as os_path
from typing import List

from pandas import DataFrame, concat, merge
from plotly.express import bar, pie
from plotly.graph_objects import Table, Figure, Indicator

path.append("../../../")
from componentes.artefato.arquivo import Arquivo
from componentes.container.container_silver import ContainerSilver
from componentes.container.categoria_de_armazenamento import \
    CategoriaDeArmazenamento
from notebooks.silver.paic.schema.paic_schema import get_paic_silver_schema
from notebooks.silver.cnae.schema.cnae_schema import get_cnae_silver_schema

#### Recuperando os dados a serem validados

In [2]:
nomes_dos_arquivos_paic_silver = [
    os_path.splitext(p=nome)[0]
    for nome
    in listdir(path="../../../dados/silver/paic")
]
dataframes = []
for nome in nomes_dos_arquivos_paic_silver:
    arquivo = Arquivo(
        nome=nome,
        categoria_de_armazenamento=CategoriaDeArmazenamento.SILVER,
        fonte="paic",
        schema=get_paic_silver_schema())
    dataframe = ContainerSilver().extrair_dados(artefato=arquivo)
    dataframe["pesquisa"] = nome
    dataframes.append(dataframe)
dataframe_paic = concat(objs=dataframes)

In [3]:
arquivo_cnae = Arquivo(
    nome="construcao",
    categoria_de_armazenamento=CategoriaDeArmazenamento.SILVER,
    fonte="cnae",
    schema=get_cnae_silver_schema())
dataframe_cnae = ContainerSilver().extrair_dados(artefato=arquivo_cnae)

#### Total de linhas

In [4]:
figura = Figure()
figura.add_trace(Indicator(value=dataframe_paic.shape[0]))
figura.update_layout(width=300, height=200, title="Total de linhas")

#### Total de linhas por pesquisa

In [5]:
total_de_linhas_por_pesquisa = (
    dataframe_paic
    .groupby(by=["pesquisa"])["pesquisa"]
    .count()
    .to_frame(name="total_de_linhas")
    .reset_index())
grafico = bar(
    data_frame=total_de_linhas_por_pesquisa,
    x="total_de_linhas",
    y="pesquisa",
    width=600,
    height=400,
    title="Total de linhas por pesquisa",
    text_auto=True)
grafico.show()

#### Total de valores nulos por coluna

In [6]:
total_de_nulos_por_colunas = (
    dataframe_paic
    .isnull()
    .sum()
    .to_frame(name="total_de_nulos")
    .reset_index()
    .rename(columns={"index": "colunas"}))
table = Table(
    header=dict(values=["colunas", "total_de_nulos"]),
    cells=dict(values=[
        total_de_nulos_por_colunas.colunas,
        total_de_nulos_por_colunas.total_de_nulos]))
figura = Figure(data=[table])
figura.update_layout(
    width=600,
    height=400,
    title="Total de valores nulos por coluna")
figura.show()

#### Relação de linhas duplicadas

In [7]:
dataframe_paic["linha_duplicada"] = (
    dataframe_paic
    .duplicated(
        subset=[
            "ano_pesquisa", "variavel", "codigo_cnae",
            "categoria_empresa", "valor", "unidade", "pesquisa"
        ]))
relacao_de_duplicidade = (
    dataframe_paic
    .groupby(by=["linha_duplicada"])["linha_duplicada"]
    .count()
    .to_frame(name="count")
    .reset_index())
relacao_de_duplicidade["linha_duplicada"] = (
    relacao_de_duplicidade["linha_duplicada"]
    .replace(to_replace={False: "Não é duplicado", True: "É duplicado"}))
grafico = pie(
    data_frame=relacao_de_duplicidade,
    values="count",
    names="linha_duplicada",
    width=600,
    height=400,
    hole=0.3,
    title="Relação de linhas duplicadas")
grafico.update_traces(textposition="inside")
grafico.show()

#### Relação de integridade do código CNAE nas pesquisas PAIC

In [8]:
relacao_codigo_cnae_integro = (
    dataframe_paic
    .merge(right=dataframe_cnae, on="codigo_cnae", how="left", indicator=True)
    .rename(columns={"_merge": "relacao"})
    .groupby(by=["relacao"])["relacao"]
    .count()
    .to_frame(name="count")
    .reset_index())
grafico = pie(
    data_frame=relacao_codigo_cnae_integro,
    values="count",
    names="relacao",
    width=600,
    height=400,
    hole=0.3,
    title="Relação de integridade do código CNAE nas pesquisas PAIC")
grafico.update_traces(textposition="inside")
grafico.show()